In [1]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
import os
from skimage import io, img_as_ubyte

In [2]:
classes =  ['flat_ng', 'flat_ok', 'hollow_nut_ng', 'hollow_nut_ok', 'plate_nut_ng', 'plate_nut_ok']
main_path = "C:/Users/000501/Desktop/Python/work/thk gifu/train/"

In [5]:
def generate_point(annotation_path = ''):
    labels = []
    points = []
    with open(annotation_path, "r") as file:    
        for line in file:
            label, lis = line.split()[0], line.split()[1:]
            labels.append(label)
            lis = list(map(float, lis))
            for i in range(len(lis)):
                lis[i] = int(lis[i] * 640)
            newlis = []
            i = 0
            while (i < len(lis)):
                newlis.append((lis[i], lis[i+1]))
                i+=2
            points.append(newlis)
    return labels, points

In [11]:
train_img_path = main_path + "images"
train_labels_path = main_path + "labels"

images_file_list = []
labels_file_list = []

for im in os.listdir(train_img_path):
    images_file_list.append(os.path.join(train_img_path, im))
    
for l in os.listdir(train_labels_path):
    labels_file_list.append(os.path.join(train_labels_path, l))
    
i = 0
n = len(labels_file_list)
class_length = len(classes)

print(n)

train_mask_path = main_path + "train_masks/"
train_image_path = main_path + "train_image/"

os.makedirs(train_mask_path, exist_ok = True)
os.makedirs(train_image_path, exist_ok = True)

for cls in classes:
    class_mask_path = train_mask_path + cls
    os.makedirs(class_mask_path, exist_ok= True)
# n = 3


171


In [ ]:
while i < n:
    label_path = labels_file_list[i]
    labels, points = generate_point(label_path)
    len_labels = len(labels)
    image_path = images_file_list[i]
    
    src = cv2.imread(image_path)
    src = cv2.resize(src, (640, 640))
    
    for cl in range(class_length):
        # print(cl)
        new_mask_path = "%s%s/image_%s.png" %(train_mask_path, classes[cl],i) 
        
        mask = np.zeros_like(src)
        
        for l, lab in enumerate(labels):
            lab = int(lab)
            if(cl == lab):
                # print(f"No - {i}")
                # print(f"Points - {points[i]}")
                contours = np.array(points[l], dtype = np.int32)
                cv2.drawContours(mask, [np.array(contours)], -1, (255, 255, 255), -1)
                    
        io.imsave(new_mask_path, mask)
        
        print(f"mask path - {new_mask_path}")
            
    new_image_path = "%simage_%s.png" %(train_image_path, i)       
    print(f"new image path - {new_image_path}") 
    io.imsave(new_image_path, src)
    
    i = i+ 1